# Merge Merchant and Consumer Features

This notebook merges consumer and merchant data into the transaction records to create a complete dataset. Also converts postcode to SA2 codes to prepare for merging with external datasets in the next step.

## Convert Postcode to SA2

In [1]:
# baseline imports
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import pyarrow as pa

In [ ]:
RAW_EXT = Path("../data/raw/external_dataset")
CLE = Path("../data/cleaned")
CUR = Path("../data/curated")

consumer_df = pd.read_parquet(CUR / "consumer_features.parquet")
merchant_df = pd.read_parquet(CUR / "merchant_features.parquet")
transactions = pd.read_parquet(CLE / "transaction.parquet")

# Postcode to SA2 concordance table
col_types = {"POSTCODE": str, "SA2_CODE_2021":str, "RATIO_FROM_TO": float}
postcode_sa2_df = pd.read_excel(RAW_EXT / "CG_POSTCODE_2021_SA2_2021.xlsx", converters=col_types)

In [3]:
# Find how many rows has a missing postcode
null_ids = int(postcode_sa2_df["POSTCODE"].isna().sum())
null_ids

120

In [4]:
# Drop rows with missing postcodes 
postcode_sa2_df = postcode_sa2_df.dropna(subset=["POSTCODE"])  

In [5]:
# For each postcode, keep the SA2 with the largest area share
sa2_ratio = (postcode_sa2_df.groupby("POSTCODE")["RATIO_FROM_TO"]
                                .max()
                                .reset_index())

# Merge back to get the corresponding SA2 code & name
preferred_sa2 = postcode_sa2_df.merge(sa2_ratio,on=["POSTCODE", "RATIO_FROM_TO"])

# Keep only needed columns, rename for consistency
sa2_poa_codes = preferred_sa2.rename(
    {
        "POSTCODE": "postcode",
        "SA2_CODE_2021": "SA2_code",
        "SA2_NAME_2021": "SA2_name"
    },
    axis=1
)[["postcode", "SA2_code", "SA2_name"]]

In [6]:
sa2_poa_codes.head(10)

,postcode,SA2_code,SA2_name
0,0800,701011002,Darwin City
1,0810,701021021,Lyons (NT)
2,0812,701021019,Karama
3,0820,701011008,Stuart Park
4,0822,702041063,East Arnhem
5,0828,701021012,Berrimah
6,0829,701031031,Howard Springs
7,0830,701041038,Durack - Marlow Lagoon
8,0832,701041043,Rosebery - Bellamack
9,0834,701031034,Virginia


## Add SA2 to consumer dataset

In [7]:
print(consumer_df.shape)

(24081, 12)


In [8]:
consumer_sa2 = consumer_df.merge(
    sa2_poa_codes,
    on="postcode",
    how="left"
)

In [9]:
# Count rows where SA2 is null
null_count = consumer_sa2['SA2_code'].isna().sum()
print(f"Number of rows with null SA2: {null_count}")

Number of rows with null SA2: 3970


Some postcodes are only for PO Boxes and Large Volume Receiver. LVR is when many large businesses, government departments and other institutions receiving high volumes of mail had their own postcode. We decided to exclude these from our analysis.

Data is taken from - https://www.matthewproctor.com/australian_postcodes#downloadlinks

In [10]:
# Define PO Box / LVR postcode ranges
po_box_ranges = [
    ("1000", "1999"), ("0200", "0299"), ("8000", "8999"),
    ("9000", "9999"), ("5800", "5999"), ("6800", "6999"),
    ("7800", "7999"), ("0900", "0999")
]

# Flag PO Box postcodes
def is_po_box(pc):
    return any(lower <= pc <= upper for lower, upper in po_box_ranges)

consumer_sa2["is_po_box"] = consumer_sa2["postcode"].apply(is_po_box)

# Split missing SA2 into PO/LVR vs not
no_sa2_mask = consumer_sa2["SA2_code"].isna()
no_sa2_but_po = consumer_sa2[ no_sa2_mask &  consumer_sa2["is_po_box"]]
no_sa2_not_po = consumer_sa2[ no_sa2_mask & ~consumer_sa2["is_po_box"]]

print("Counts:", {
    "no_sa2_but_po": len(no_sa2_but_po),
    "no_sa2_not_po": len(no_sa2_not_po)
})
print("\nPO/LVR postcodes with no SA2:", np.sort(no_sa2_but_po["postcode"].unique()))
print("\nNon-PO postcodes with no SA2:", np.sort(no_sa2_not_po["postcode"].unique()))

Counts: {'no_sa2_but_po': 3111, 'no_sa2_not_po': 859}

PO/LVR postcodes with no SA2: ['0200' '0906' '0907' '0909' '1001' '1002' '1003' '1004' '1005' '1006'
 '1007' '1008' '1009' '1010' '1020' '1021' '1022' '1023' '1025' '1026'
 '1027' '1028' '1029' '1030' '1031' '1032' '1033' '1034' '1035' '1036'
 '1037' '1038' '1039' '1040' '1041' '1042' '1043' '1044' '1045' '1046'
 '1100' '1101' '1105' '1106' '1107' '1108' '1109' '1110' '1112' '1113'
 '1114' '1115' '1116' '1117' '1118' '1119' '1120' '1121' '1122' '1123'
 '1124' '1125' '1126' '1127' '1128' '1129' '1130' '1131' '1132' '1133'
 '1134' '1135' '1136' '1137' '1138' '1139' '1140' '1141' '1142' '1143'
 '1144' '1145' '1146' '1147' '1148' '1149' '1150' '1151' '1152' '1153'
 '1154' '1155' '1156' '1157' '1158' '1159' '1160' '1161' '1162' '1163'
 '1164' '1165' '1166' '1167' '1168' '1169' '1170' '1171' '1172' '1173'
 '1174' '1175' '1176' '1177' '1178' '1179' '1180' '1181' '1182' '1183'
 '1184' '1185' '1186' '1187' '1188' '1189' '1190' '1191' '1192'

In [11]:
# Total before filtering
total_consumers = consumer_sa2["consumer_id"].nunique()

# Count removals
removed_po_box = consumer_sa2[consumer_sa2["is_po_box"]]
removed_unknown_sa2 = consumer_sa2[(consumer_sa2["SA2_code"].isna())]

n_removed_po_box = removed_po_box["consumer_id"].nunique()
n_removed_unknown = removed_unknown_sa2["consumer_id"].nunique()

# Combined removal (avoid double-counting)
removed_combined = consumer_sa2[
    consumer_sa2["is_po_box"] | consumer_sa2["SA2_code"].isna()]
n_removed_total = removed_combined["consumer_id"].nunique()

pct_removed_total = n_removed_total / total_consumers * 100

print(f"Total consumers: {total_consumers}")
print(f"Removed (PO Box/LVR): {n_removed_po_box}")
print(f"Removed (Unknown SA2): {n_removed_unknown}")
print(f"Total removed: {n_removed_total} ({pct_removed_total:.2f}%)")
print(f"Remaining consumers: {total_consumers - n_removed_total}")

Total consumers: 24081
Removed (PO Box/LVR): 3117
Removed (Unknown SA2): 3970
Total removed: 3976 (16.51%)
Remaining consumers: 20105


We decided to remove all consumers whose postcodes fall within **PO box** or **LVR (Large Volume Receiver)** ranges. These addresses typically do not represent the consumer’s place of residence and therefore cannot be reliably mapped to a meaningful SA2 region. 

For the small number of remaining consumers with unknown or unmapped SA2 codes, we also excluded them from further analysis. Attempting to impute SA2 values for these consumers could introduce noise or bias, particularly because our downstream modelling relies heavily on external datasets (GCP, WPP, APRA-SEIFA) that are SA2-based. By filtering out these records early, we ensure that the features used in our models accurately reflect the socioeconomic and demographic context of the consumers we are analysing, leading to more robust and interpretable results.

In [12]:
# Drop rows
print(f"Shape before drop: {consumer_sa2.shape}")
consumer_sa2_cleaned = consumer_sa2[~(consumer_sa2["SA2_code"].isna())]
consumer_sa2_cleaned = consumer_sa2_cleaned.drop(columns=["is_po_box"])

Shape before drop: (24081, 15)


In [13]:
# Count rows where SA2 is null
null_count = consumer_sa2_cleaned['SA2_code'].isna().sum()
print(f"Number of rows with null SA2: {null_count}")
print(f"Shape after drop: {consumer_sa2_cleaned.shape}")

Number of rows with null SA2: 0
Shape after drop: (20111, 14)


In [14]:
consumer_sa2_cleaned.head(5)

,consumer_id,transaction_count,total_spend,unique_merchants,avg_spend,fraud_prob_avg,name,address,state,postcode,gender,user_id,SA2_code,SA2_name
0,1000031,576,94976.245070,344,164.889314,0.246955,Pamela Olsen,024 Hood Summit Suite 983,NSW,2177,Female,12606,127021509,Bonnyrigg Heights - Bonnyrigg
1,1000051,561,93671.439271,342,166.972263,0.097455,Calvin Scott,5027 Richard Lane Apt. 564,QLD,4053,Male,8622,302011025,Everton Park
2,1000067,552,72193.558355,344,130.785432,0.126411,Michelle Fox,6749 Peter Cape,WA,6623,Female,2917,511041291,Morawa
3,1000092,577,88144.192996,339,152.762899,NaN,Chelsea Cook,5495 Jones Light,QLD,4850,Female,12179,318011465,Ingham Surrounds
4,1000115,583,85280.337277,355,146.278452,0.104338,Troy Hess,788 Kristopher Crest,WA,6030,Male,12071,505031105,Mindarie - Quinns Rocks - Jindalee


In [15]:
consumer_sa2_cleaned.to_csv(CUR / "consumer_sa2.csv")

## Merge consumer, merchant and transactions

In [16]:
# Rename columns to avoid confusion
consumer_sa2 = consumer_sa2.rename(columns={"fraud_prob_avg": "consumer_fraud_prob_avg"})
consumer_sa2 = consumer_sa2.rename(columns={"name": "consumer_name"})

merchant_df = merchant_df.rename(columns={"name": "merchant_name"})

In [17]:
# Join consumer features into transactions
tx_cons = transactions.merge(
    consumer_sa2,
    on="user_id",
    how="left"
)

In [18]:
# Join merchant features into transactions
tx_full = tx_cons.merge(
    merchant_df,
    on="merchant_abn",
    how="left"
)

In [19]:
tx_full.head()

,user_id,merchant_abn,dollar_value,order_id,order_datetime,consumer_id,transaction_count,total_spend,unique_merchants,avg_spend,...,tags,categories,type,take_rate,order_count,total_sales,avg_order_value,unique_consumers,repeat_consumers,repurchase_rate
0,1,28000487688,133.226894,0c37b3f7-c7f1-48cb-bcc7-0a58e76608ea,2021-02-28,1195503,553,79177.745051,324,143.178562,...,"((books, periodicals, anD newspapers), (b), (t...","[books, periodicals, and newspapers]",b,4.24,3791,9.270055e+05,244.527973,3495,277,0.079256
1,18485,62191208634,79.131400,9e18b913-0465-4fd4-92fd-66d15e65d93c,2021-02-28,1212819,554,106704.069885,329,192.606624,...,"[(furniture, home furnishings and equipment sh...","[furniture, home furnishings and equipment sho...",c,2.17,16380,1.423276e+06,86.891108,11863,3578,0.301610
2,1,83690644458,30.441348,40a2ff69-ea34-4657-8429-df7ca957d6a1,2021-02-28,1195503,553,79177.745051,324,143.178562,...,"[(gift, card, novelty, and souvenir shops), (b...","[gift, card, novelty, and souvenir shops]",b,3.15,35852,3.183224e+06,88.787899,18665,10550,0.565229
3,18488,39649557865,962.813341,f4c1a5ae-5b76-40d0-ae0f-cb9730ac325a,2021-02-28,1302316,590,84946.035616,342,143.976332,...,"([artist supply and craft shops], [c], [take r...",[artist supply and craft shops],c,1.47,21919,9.857402e+06,449.719528,14462,5494,0.379892
4,2,80779820715,48.123977,cd09bdd6-f56d-489f-81ea-440f4bda933c,2021-02-28,179208,567,116325.665308,345,205.159904,...,"([watch, clock, and jewelry repair shops], [b]...","[watch, clock, and jewelry repair shops]",b,4.71,36438,1.272197e+06,34.914021,18807,10786,0.573510


In [20]:
tx_full.shape

(13614672, 30)

In [21]:
# count NaN values per column
nan_counts = tx_full.isna().sum()
nan_counts

user_id                          0
merchant_abn                     0
dollar_value                     0
order_id                         0
order_datetime                   0
consumer_id                      0
transaction_count                0
total_spend                      0
unique_merchants                 0
avg_spend                        0
consumer_fraud_prob_avg    2234384
consumer_name                    0
address                          0
state                            0
postcode                         0
gender                           0
SA2_code                   2244225
SA2_name                   2244225
is_po_box                        0
merchant_name                    0
tags                             0
categories                       0
type                             0
take_rate                        0
order_count                      0
total_sales                      0
avg_order_value                  0
unique_consumers                 0
repeat_consumers    

In [22]:
# Save to curated folder
tx_full.to_parquet(CUR / "all_given_data.parquet", index=False)

## Summary

**all_given_data:**

-----Transaction-----
- **user_id** : used to merge consumer features
- **merchant_abn** : used to merge merchant features
- **dollar_value**
- **order_id**
- **order_datetime**

-----Consumer features----- 
- **consumer_id** → unique identifier for the consumer  
- **transaction_count** → total number of transactions made  
- **total_spend** → total dollar value spent across all transactions  
- **unique_merchants** → number of distinct merchants purchased from  
- **avg_spend** → average spend per transaction  
- **consumer_fraud_prob_avg** → average fraud probability across transactions  
- **consumer_name** → consumer’s name  
- **address** → consumer’s address  
- **state** → state of residence  
- **postcode** → postcode of residence  
- **gender** → gender of the consumer 
- **SA2_code** → SA2 code of the consumer 
- **SA2_name** → SA2 name of the consumer 

-----Merchant features----- 
- **merchant_name** → merchant’s name  
- **tags** → descriptive tags associated with the merchant  
- **categories** → category labels for the merchant  
- **type** → merchant type classification  
- **take_rate** → BNPL commission rate (%)  
- **order_count** → total number of transactions with this merchant     
- **total_sales** → total revenue generated from transactions  
- **avg_order_value** → average value per transaction    
- **unique_consumers** → number of distinct customers served  
- **repeat_consumers** → number of customers with 2+ purchases  
- **repurchase_rate** → proportion of customers who are repeat buyers 